# Vision Transformer系列SOTA模型演进及其在MindCV最佳实践

在计算机视觉领域， 卷积神经网络（Convolutional Neural Network， CNN)是一种被广泛使用的网络结构。但是，随着self-attention结构在自然语言处理（Natural Language Processing, NLP) 任务上逐渐占据优势地位， Transformer这个在NLP领域大放异彩的模型，也终于进入了计算机视觉领域。 2020年10月，Google 的文章 An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale 将Transformer运用到图像分类任务上，提出了Vision Transformer 模型，简称ViT。 

如果你还不太熟悉self-attention 以及Transformer， 可以阅读[这篇文章](https://zhuanlan.zhihu.com/p/345680792)。本文将主要介绍ViT系列模型结构的演进历史，ViT的源代码，以及如何将ViT运用到图像识别的实战例子。

本文的阅读时长大约15分钟。

* [ViT 系列模型结构演进历史](#first-sec)
    * [ViT](#1.1-subsec)
    * [ConViT](#1.2-subsec)
    * [CrossViT](#1.3-subsec)
    * [PiT](#1.4-subsec)
    * [MobileViT](#1.5-subsec)
* [ViT 实战](#second-sec)

## ViT 系列模型结构演进历史 <a class="anchor" id="first-sec"></a>

### ViT <a class="anchor" id="1.1-subsec"></a>
ViT[1]的核心思想是像处理NLP任务中的sequence of tokens一样来处理图像信息：首先将图像分割成一个个的图像块，然后将图像块转换成向量， 最后用Transformer来处理向量的序列。



<p>
<img src="./images/ViT_architecture.PNG" alt="vit" width="700"/>
<em><center>The architecture of ViT. Image source [1]. </center></em>
</p>



ViT的第一个步骤是**从输入的二维图像得到Patch Embedding的过程**。 Patch Embedding可以理解为视觉的“单词”。 得到Patch Embedding的过程是将2维图像的信息用一个由多个1维向量组成的序列来表达的过程。这是因为Transformer不能直接处理2维的图像输入。详细的过程如下。

首先我们要将2维图像均匀的分割成$P\times P$大小的小块，总共分割成$N$块。经过分割处理后的2维patches形状是$(N, P^2, C)$, 其中$C$ 是图像的channel数量，通常为3（RGB图像）。经过flatten处理后， 2维patches 变为1维的序列，形状是$(N, P^2\cdot C)$。

举例来说，当输入是一张$224\times 224$的RGB图片时，如果分割出的图像块大小为$16\times 16$, 图像块的数量就是$(224/16)^2=196$。经过flatten 后的输入变为$(196, 768)$ ($16\times 16\times 3 = 768$)。

接下来，我们将1维序列输入到线性投射层中。线性投射层可以将输入的向量们映射到一个固定的维度$D$。 至此，我们已经从一张2维图像得到了由一个1维向量组成的序列$(N, D)$。除了Patch Embedding之外，ViT还使用了一个特殊的embedding $[cls]$与Patch Embedding连接在一起。 $[cls]$ embedding是可以学习的， 相当于一个全局的特征embedding，能够让Transformer encoder在 $[cls]$ 位置输出图像分类的结果。 因此，序列的长度从$N$ 增加到$N+1$。

ViT的第二个步骤是**对Patch Embedding 添加位置信息**。ViT使用了NLP中Transformer常用的learnable 1D position embeddings （可学习的1维位置编码）。通过将position embedding和patch embedding相加，我们得到了添加位置信息的patch embedding。

接下来， ViT的第三个步骤是**使用Transformer对输入序列进行处理**。 ViT中的Transformer只包含encoder layers, 也就是$L$层 `LayerNorm -> Multihead Self-Attention -> LayerNorm -> MLP`的堆叠。 这个过程可以利用Transformer的自注意力机制来学习图像中不同位置的信息之间的关系，从而提高对图像的理解能力。 Transformer encoder layers不会改变输入的形状大小，因此经过若干encoder layers之后，Transformer输出的形状大小为$(N+1, D)$. 

ViT的最后步骤是**获取图像分类结果**。前面说到Transformer在$[cls]$ embedding所在位置的输出，对应着图像分类的结果。通常$[cls]$ embedding的位置是第一个。我们把经过L层encoder layers处理后的输出记为$z_L$, 那么$z_L^0$就是位置为0的输出向量。最后ViT输出的图像分类结果由$z_L^0$经过一层LayerNorm得到：$\hat{y} = LN(z_L^0)$。


与CNN相比，ViT的显著特点是，在有足够多的数据进行预训练的前提下，ViT的表现能够超过CNN; 反之，ViT的表现则不如同等大小的ResNets。这是由于CNN的归纳偏置（主要是局部性和平移不变性）可以作为一种先验知识，让CNN只需要使用较少的训练数据就得到比较好的结果。

[1]: Dosovitskiy, A., Beyer, L., Kolesnikov, A., Weissenborn, D., Zhai, X., Unterthiner, T., Dehghani, M., Minderer, M., Heigold, G., Gelly, S., Uszkoreit, J., & Houlsby, N. (2020). An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale. ArXiv, abs/2010.11929.


接下来，我们以MindCV实现的[ViT模型代码](https://github.com/mindspore-lab/mindcv/blob/20d54a9f383a2332eb344cee748be63b0dedf437/mindcv/models/vit.py)为例子，来深入理解ViT的模型结构。下面代码中的ViT没有包含MLP Head, 但是其输出的结果可以输入任意的MLP classifier 来预测图像类别。

In [5]:
from typing import List, Optional, Union, Tuple, Dict

import numpy as np
import math
import mindspore as ms
from mindspore import Tensor, nn
from mindspore import ops
from mindspore import ops as P
from mindspore.common.initializer import Normal, initializer
from mindspore.common.parameter import Parameter

class ViT(nn.Cell):
    def __init__(
        self,
        image_size: int = 224,
        input_channels: int = 3,
        patch_size: int = 16,
        embed_dim: int = 768,
        num_layers: int = 12,
        num_heads: int = 12,
        mlp_dim: int = 3072,
        keep_prob: float = 1.0,
        attention_keep_prob: float = 1.0,
        drop_path_keep_prob: float = 1.0,
        activation: nn.Cell = nn.GELU,
        norm: Optional[nn.Cell] = nn.LayerNorm,
        pool: str = "cls",
    ) -> None:
        super().__init__()

        self.patch_embedding = PatchEmbedding(image_size=image_size,
                                              patch_size=patch_size,
                                              embed_dim=embed_dim,
                                              input_channels=input_channels) # Patch Embedding 将原始图片分割成P x P 大小的小块，返回(batch_size,N, P*P*C)
        num_patches = self.patch_embedding.num_patches

        if pool == "cls": # 使用cls pooling的方式指的是使用class token来指示class prediction的位置
            self.cls_token = init(init_type=Normal(sigma=1.0),
                                  shape=(1, 1, embed_dim),
                                  dtype=ms.float32,
                                  name="cls",
                                  requires_grad=True)
            self.pos_embedding = init(init_type=Normal(sigma=1.0),
                                      shape=(1, num_patches + 1, embed_dim),
                                      dtype=ms.float32,
                                      name="pos_embedding",
                                      requires_grad=True)
            self.concat = ops.Concat(axis=1)
        else: #否则，将Transformer encoder输出的N个embedding平均，得到的average embedding 可以输入classifier
            self.pos_embedding = init(init_type=Normal(sigma=1.0),
                                      shape=(1, num_patches, embed_dim),
                                      dtype=ms.float32,
                                      name="pos_embedding",
                                      requires_grad=True)
            self.mean = ops.ReduceMean(keep_dims=False)

        self.pool = pool
        self.pos_dropout = nn.Dropout(keep_prob)
        self.norm = norm((embed_dim,))
        self.tile = ops.Tile()
        self.transformer = TransformerEncoder(
            dim=embed_dim,
            num_layers=num_layers,
            num_heads=num_heads,
            mlp_dim=mlp_dim,
            keep_prob=keep_prob,
            attention_keep_prob=attention_keep_prob,
            drop_path_keep_prob=drop_path_keep_prob,
            activation=activation,
            norm=norm,
        )

    def construct(self, x):
        """ViT construct."""
        x = self.patch_embedding(x)

        if self.pool == "cls":
            cls_tokens = self.tile(self.cls_token, (x.shape[0], 1, 1)) # concatentation前准备shape:在第一个dimension重复batch_size次
            x = self.concat((cls_tokens, x))
            x += self.pos_embedding
        else:
            x += self.pos_embedding
        x = self.pos_dropout(x)
        x = self.transformer(x)
        x = self.norm(x)

        if self.pool == "cls":
            x = x[:, 0]
        else:
            x = self.mean(x, (1, ))  # (1,) or (1,2)
        return x

### ConViT <a class="anchor" id="1.2-subsec"></a>

为了缓解Transformer缺乏归纳偏置的问题，从而减少对预训练数据量的要求，Facebook在2021年提出的ConViT[2]首次将"软性归纳偏置"（soft inductive bias）引入到Transformer layer中。如原论文所说，这是为了“将CNN和Transformer两种模型的强项结合到一起，并且规避这两种模型的缺点”。

CNN的归纳偏置主要体现在两个方面： 局部连接和权值共享。前者使每一个输入神经元只跟输入周围的一小部分像素相连，保证了特征的局部性；后者使不同位置的特征被相同的权重处理，保证了平移不变性。这两个方面是以硬编码的形式体现在网络结构中的，但是这种硬编码的归纳偏置在处理长距离学习的问题上存在先天的劣势。Transformer在处理输入序列时，可以通过self-attention学习任意距离的两个输入之间的attention。 但是缺乏先验知识，在图像任务上，Transformer需要更多的预训练数据。

ConViT的作者提出的gated positional self-attention(GPSA)能够将CNN和Transformer的优势结合起来，既可以利用CNN的对局部特征的学习能力和采样效率，又能够利用Transformer对全局关系的学习能力和灵活性。GPSA通过在self-attention中引入一个门控参数，来调节对局部特征和全局信息的关注程度。


<p>
<img src="./images/ConViT_architecture.PNG" alt="convit" width="550"/>
<em><center>The architecture of ConViT (GPSA: gated positional self-attention; SA: self-attention). Image source [2]. </center></em>
</p>



ConViT整体的结构与ViT类似，只是将ViT的头几层encoder layer中的SA替换成了GPSA。 GPSA的结构如上图的右半部分所示。其中，$\lambda$ 是一个可学习的门控参数。 $\sigma(\lambda)$ 是经过sigmoid函数后的门控参数，将GPSA的计算分为左右两个分支。

左分支类似于self-attention中attention weights的计算过程： $X_i$ 和$X_j$ 分别经过$W_{qry}$和$W_{key}$的映射得到$Q_i = W_{qry}X_i$ 和$W_j = W_{key}X_j$。 我们计算他们之间的点积并经过softmax函数计算attention weights, 等于$softmax(Q_iW_j^T)$。考虑到多头注意力的结构，第$h$个head得到的attention weights就是$softmax(Q_i^h(W_j^h)^T)$。

在右分支，作者使用了positional self-attention (PSA)[3]的结构， 其核心思想是在self-attention引入不同像素之间相对位置的信息。$r_{ij}\in R^{D_{pos}}$ 编码的信息就是像素$i$和像素$j$的相对位置信息。在ConViT训练过程中，$r_{ij}$是固定不变的。$v_{pos}^h \in R^{D_{pos}}$代表着第$h$个head学到的相对位置注意力。在论文中， $D_{pos}$大小为3。通常$D_{pos}<< D$ 这说明引入positional self-attention产生的多余计算量非常小，几乎可以忽略。在右分支部分，第$h$个head得到的positional attention weights就是$softmax((v_{pos}^h)^T r_{ij})$。

最后，通过门控参数$\lambda$将两部分attention weights 结合起来, 得到$A_{ij} = (1-\sigma(\lambda))softmax(Q_i^h(W_j^h)^T) + \sigma(\lambda)softmax((v_{pos}^h)^T r_{ij})$。

某一层GPSA layer的输出结果应该是：
$GPSA^h(X) := normalize(A^h)XW^T_{val}$, 其中$W_{val}$ 表示value 的映射矩阵。

[2]: d'Ascoli, S., Touvron, H., Leavitt, M.L., Morcos, A.S., Biroli, G., & Sagun, L. (2021). ConViT: improving vision transformers with soft convolutional inductive biases. Journal of Statistical Mechanics: Theory and Experiment, 2022.

[3]: Ramachandran, P., Parmar, N., Vaswani, A., Bello, I., Levskaya, A., & Shlens, J. (2019). Stand-Alone Self-Attention in Vision Models. ArXiv, abs/1906.05909.

由于ConViT的主体结构跟ViT非常类似，我们主要分析MindCV中[GPSA layer的实现代码](https://github.com/mindspore-lab/mindcv/blob/main/mindcv/models/convit.py#L68):

In [12]:
class GPSA(nn.Cell):
    def __init__(
        self,
        dim: int,
        num_heads: int,
        qkv_bias: bool = False,
        attn_drop: float = 0.0,
        proj_drop: float = 0.0,
    ) -> None:
        super().__init__()

        self.dim = dim
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim**-0.5

        self.q = nn.Dense(in_channels=dim, out_channels=dim, has_bias=qkv_bias) # query matrix
        self.k = nn.Dense(in_channels=dim, out_channels=dim, has_bias=qkv_bias) # key matrix
        self.v = nn.Dense(in_channels=dim, out_channels=dim, has_bias=qkv_bias) # value matrix

        self.attn_drop = nn.Dropout(keep_prob=1.0 - attn_drop)
        self.proj = nn.Dense(in_channels=dim, out_channels=dim)
        self.pos_proj = nn.Dense(in_channels=3, out_channels=num_heads) # 相当于v^h_{pos} 
        self.proj_drop = nn.Dropout(keep_prob=1.0 - proj_drop)
        self.gating_param = Parameter(ops.ones((num_heads), ms.float32))# 每一个 head都有一个门控参数
        self.softmax = nn.Softmax(axis=-1)
        self.batch_matmul = ops.BatchMatMul()
        self.rel_indices = get_rel_indices() # 相对位置信息编码r_{ij}，在训练中不会变化

    def construct(self, x: Tensor) -> Tensor:
        """ConViT construct."""
        B, N, C = x.shape
        attn = self.get_attention(x)
        v = ops.reshape(self.v(x), (B, N, self.num_heads, C // self.num_heads))
        v = ops.transpose(v, (0, 2, 1, 3))
        x = ops.transpose(self.batch_matmul(attn, v), (0, 2, 1, 3))
        x = ops.reshape(x, (B, N, C))
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

    def get_attention(self, x: Tensor) -> Tensor:
        B, N, C = x.shape
        q = ops.reshape(self.q(x), (B, N, self.num_heads, C // self.num_heads))
        q = ops.transpose(q, (0, 2, 1, 3))
        k = ops.reshape(self.k(x), (B, N, self.num_heads, C // self.num_heads))
        k = ops.transpose(k, (0, 2, 3, 1))
        # 右分支的positional attention weights
        pos_score = self.pos_proj(self.rel_indices)
        pos_score = ops.transpose(pos_score, (0, 3, 1, 2))
        pos_score = self.softmax(pos_score)
        # 左分支的self-attention weights
        patch_score = self.batch_matmul(q, k)
        patch_score = ops.mul(patch_score, self.scale)
        patch_score = self.softmax(patch_score)

        gating = ops.reshape(self.gating_param, (1, -1, 1, 1))
        gating = ops.Sigmoid()(gating)
        attn = (1.0 - gating) * patch_score + gating * pos_score # 门控参数控制weighted sum
        attn = self.attn_drop(attn)
        return attn


### CrossViT <a class="anchor" id="1.3-subsec"></a>

MIT 在2021年提出的CorssViT 针对的是ViT如何学习多尺度图像特征的问题。 为了适应不同尺度的物体， CrossViT采取了双分支结构，可以分别处理较小尺寸的image patches和较大尺寸的image patches。在两个分支的encoder layers对不同尺度的image patches处理完以后，再通过cross-attention layers进行信息融合。具体的模型结构如下图所示：


<p>
<img src="./images/CrossViT_architecture.PNG" alt="vit" width="400"/>
<em><center>The architecture of CrossViT. Image source [4]. </center></em>
</p>

CrossViT的主要贡献在于cross-attention layer的设计。作者希望融合多尺度特征的信息，但是要避免将特征简单拼接起来造成计算量过大。$cls$这个特殊的embedding某种程度上代表了全局特征（因为可以据此得到class prediction), 所以作者提出，将某一个分支的cls embedding与另一个分支的patches embedding做cross-attention就能融合两个分支的信息。 这样做的好处是没有造成特征维度的增加，因此节省了计算量。



<p>
<img src="./images/CrossViT_cross_attention.PNG" alt="vit" width="400"/>
<em><center>The architecture of CrossViT cross-attention layer for the Large Branch. Image source [4]. </center></em>
</p>


上图展示的是较大尺寸的image patches对应的分支（记为Large Branch）中的cross-attention layer。 提取Large Branch 中的cls embedding, 经过维度转换函数$f^l(\cdot)$得到$x^{l \prime}_{cls}$。首先$x^{l \prime}_{cls}$ 作为query embedding 与$W_q$ 计算乘积， 然后$x^{l \prime}_{cls}$ 与来自Small Branch的image patches emebdding拼接在一起，分别于$W_k$和$W_v$计算乘积。剩下的计算和Transformer 中的cross-attention类似，只是在输出结果前还要经过另一个维度转换函数$g^l(\cdot)$， 从而保证$y^{l \prime}_{cls}$与$x^{l}_{patch}$ 维度的一致性。上图的输出结果还可以作为下一层cross-attention layer的输入，持续处理两个分支的信息融合。

[4]: Chen, C., Fan, Q., & Panda, R. (2021). CrossViT: Cross-Attention Multi-Scale Vision Transformer for Image Classification. 2021 IEEE/CVF International Conference on Computer Vision (ICCV), 347-356.


CrossViT的代码相对比较复杂，这里只截取了一部分与cross-attention layer 图片红圈内有关的代码， 体现了跨分支信息的融合。更多代码可以参考[这里](https://github.com/mindspore-lab/mindcv/blob/main/mindcv/models/crossvit.py)。

In [11]:
class CrossAttention(nn.Cell):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.wq = nn.Dense(dim, dim, has_bias=qkv_bias)
        self.wk = nn.Dense(dim, dim, has_bias=qkv_bias)
        self.wv = nn.Dense(dim, dim, has_bias=qkv_bias)
        self.attn_drop = nn.Dropout(1.0 - attn_drop)
        self.proj = nn.Dense(dim, dim)
        self.proj_drop = nn.Dropout(1.0 - proj_drop)

    def construct(self, x: Tensor) -> Tensor:
        B, N, C = x.shape  # x是concat((x^{l \prime}_{cls}, {x_{patch}^s}))的结果，或者是concat((x^{s \prime}_{cls}, {x_{patch}^l}))的结果
        q = self.wq(x[:, 0:1, ...]).reshape(B, 1, self.num_heads, C // self.num_heads) # cls embedding 是x[:, 0:1, ...]
        q = ops.transpose(q, (0, 2, 1, 3))  # queries

        k = self.wk(x).reshape(B, N, self.num_heads, C // self.num_heads)
        k = ops.transpose(k, (0, 2, 1, 3))   #keys

        v = self.wv(x).reshape(B, N, self.num_heads, C // self.num_heads)
        v = ops.transpose(v, (0, 2, 1, 3))   # values
 
        batchmatual = ops.BatchMatMul(transpose_b=True)
        attn = batchmatual(q, k) * self.scale
        softmax = nn.Softmax()
        attn = softmax(attn)
        attn = self.attn_drop(attn) # attention weights after dropout
        batchmatual2 = ops.BatchMatMul()
        x = batchmatual2(attn, v)
        x = ops.transpose(x, (0, 2, 1, 3))
        x = x.reshape(B, 1, C)
        x = self.proj(x)
        x = self.proj_drop(x) 

        return x


### PiT <a class="anchor" id="1.4-subsec"></a>

Pooling-based ViT （PiT）针对的问题同样是图像（或者是feature map）的尺寸问题。不同于CrossViT采取双分支的方式来处理两种不同尺度的输入，PiT的策略与CNN类似，都是从一个较大尺寸的图像出发，渐进地缩小其空间尺寸并且增大特征维度。如原文所说，这样做的好处是能够增加模型的表达能力和泛化能力(expressiveness and generalization)。


<p>
<img src="./images/Pit_dimension_config.PNG" alt="vit" width="800"/>
<em><center>The dimension configuration of ResNet50, ViT, and PiT. Image source [5]. </center></em>
</p>

ResNet50 由若干个卷积模块组成，每一个模块都会将特征图的空间尺寸缩小并将特征维度增加。但是，ViT从得到image patches (形状是$(N, P^2\times C)$，$P$在图中等于14，$C$在图中等于384)， 代表着空间尺寸的$P^2$不会随着self-attention layer的数量增加而变化。同样的， 特征维度$C$也不会随着self-attention layer的数量增加而变化。

PiT 设计了由几层self-attention layer构成的block。每一个block能将输入特征的空间尺寸减半，将输入特征的维度增加到两倍。$cls$ embedding的维度也相应地增加，保证其与patch embedding的维度能够对齐。这样的操作是通过每一个block中的Pooling layer 来完成的。


<p>
<img src="./images/Pit_pooling_layers.PNG" alt="vit" width="400"/>
<em><center>The pooling layer of PiT. Image source [5]. </center></em>
</p>

PiT的pooling layer首先将输入特征由$(N， P^2\times C)$的一维特征reshape成为$(N, P, P, C)$的三维特征。在这之后，可以使用depth-wise convolution将特征图的空间尺寸缩小，将其特征维度增大。

[5]: Heo, B., Yun, S., Han, D., Chun, S., Choe, J., & Oh, S. (2021). Rethinking Spatial Dimensions of Vision Transformers. 2021 IEEE/CVF International Conference on Computer Vision (ICCV), 11916-11925.


下面的代码就是MindCV实现的PiT Pooling layer。代码的其余部分参见[这里](https://github.com/mindspore-lab/mindcv/blob/main/mindcv/models/pit.py)。

In [3]:
class conv_head_pooling(nn.Cell):
    """define pooling layer using conv in spatial tokens with an additional fully-connected layer
    (to adjust the channel size to match the spatial tokens)"""

    def __init__(
        self,
        in_feature: int,
        out_feature: int,
        stride: int,
        pad_mode: str = "pad",
    ) -> None:
        super().__init__()
        self.conv = nn.Conv2d(
            in_feature,
            out_feature,
            kernel_size=stride + 1,
            padding=stride // 2,
            stride=stride,
            pad_mode=pad_mode,
            group=in_feature,
            has_bias=True,
        )
        self.fc = nn.Dense(in_channels=in_feature, out_channels=out_feature, has_bias=True)

    def construct(self, x, cls_token):
        x = self.conv(x) # x是经过reshape后的feature embedding
        cls_token = self.fc(cls_token) # self.fc 用于改变cls token的embedding dimension

        return x, cls_token


### MobileViT  <a class="anchor" id="1.5-subsec"></a>

2021年， 苹果公司提出MobileViT。 这是一种结合了CNN优点（例如空间归纳偏置）和Transformer优点（例如输入自适应加权和全局处理）的轻量级网络结构，能够在移动端设备运行并取得超过（同样数量的参数）MobileNetv3的效果。


前面提到, CNN的卷积滤波器设计在长距离学习的任务上存在着先天的劣势，但是self-attention擅长学习全局信息。因此作者提出了一种混合了CNN卷积滤波器和self-attention的特殊结构，称为MobileViT Block.


<p>
<img src="./images/MobileViT_architecture.PNG" alt="vit" width="700"/>
<em><center>The architecture of MobileViT and MobileViT Block. Image source [6]. </center></em>
</p>


MobileViT Block首先用$n\times n$的卷积滤波器学习局部的信息，再用$1\times 1$的卷积滤波器将特征维度投射到$d$。 紧接着，在特征图上分割出一共$N$个image patches, 每一个的大小为$P$, 特征维度为$d$，经过展开得到$(P, N, d)$的特征$X_U$。其中每一个像素点位置$p$对应的特征$X_U(p)\in R^{N\times d}$。将$X_U(p)$输入到Transformer可以学习关于N个image patches在$p$位置的全局信息:

$X_G(p) = Transformer(X_U(p)), 1\leq p \leq P$.

这一个过程正如下图所示。以每一个image patch的中心像素为例，通过Transformer可以学习到其他所有image patches在中心位置的全局信息（黄色箭头），而每一个中心位置的像素已经通过之前的卷积滤波器学到了周围像素的局部信息。如此一来，就实现了局部信息和全局信息的共同学习。
<p>
<img src="./images/MobileViT_local_global.PNG" alt="vit" width="200"/>
<em><center>Every pixel sees every other pixel in the MobileViT block. Image source [6]. </center></em>
</p>

经过Transformer学习之后，将特征折叠成原来的形状，并经过$1\times 1$ conv $-> concat -> n\times n $ conv 输出与原特征图形状一样的特征。

[6]: Mehta, S., & Rastegari, M. (2021). MobileViT: Light-weight, General-purpose, and Mobile-friendly Vision Transformer. ArXiv, abs/2110.02178.

下面的代码展示了MobileViT Block的结构。更多关于MobileViT的代码见[这里](https://github.com/mindspore-lab/mindcv/blob/main/mindcv/models/mobilevit.py)。

In [7]:
class MobileViTBlock(nn.Cell):

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        transformer_dim: int,
        ffn_dim: int,
        n_transformer_blocks: int = 2,
        head_dim: int = 32,
        attn_dropout: float = 0.0,
        dropout: float = 0.0,
        ffn_dropout: float = 0.0,
        patch_h: int = 8,
        patch_w: int = 8,
        conv_ksize: Optional[int] = 3,
        *args,
        **kwargs
    ) -> None:
        super().__init__()

        conv_3x3_in = ConvLayer(
            in_channels=in_channels,
            out_channels=in_channels,
            kernel_size=conv_ksize,
            stride=1
        )
        conv_1x1_in = ConvLayer(
            in_channels=in_channels,
            out_channels=transformer_dim,
            kernel_size=1,
            stride=1
        )

        conv_1x1_out = ConvLayer(
            in_channels=transformer_dim,
            out_channels=in_channels,
            kernel_size=1,
            stride=1
        )
        conv_3x3_out = ConvLayer(
            in_channels=2 * in_channels,
            out_channels=out_channels,
            kernel_size=conv_ksize,
            stride=1,
            pad_mode="pad",
            padding=1
        )

        local_rep = []
        local_rep.append(conv_3x3_in)
        local_rep.append(conv_1x1_in)
        self.local_rep = nn.SequentialCell(local_rep) # 3x3 conv -> 1x1 conv

        assert transformer_dim % head_dim == 0
        num_heads = transformer_dim // head_dim

        self.global_rep = [
            TransformerEncoder(
                embed_dim=transformer_dim,
                ffn_latent_dim=ffn_dim,
                num_heads=num_heads,
                attn_dropout=attn_dropout,
                dropout=dropout,
                ffn_dropout=ffn_dropout
            )
            for _ in range(n_transformer_blocks)
        ]
        self.global_rep.append(nn.LayerNorm((transformer_dim,)))
        self.global_rep = nn.CellList(self.global_rep) # 若干层Transformer encoder layers

        self.conv_proj = conv_1x1_out
        self.fusion = conv_3x3_out

        self.patch_h = patch_h
        self.patch_w = patch_w
        self.patch_area = self.patch_w * self.patch_h # 

        self.cnn_in_dim = in_channels
        self.cnn_out_dim = transformer_dim
        self.n_heads = num_heads
        self.ffn_dim = ffn_dim
        self.dropout = dropout
        self.attn_dropout = attn_dropout
        self.ffn_dropout = ffn_dropout
        self.n_blocks = n_transformer_blocks
        self.conv_ksize = conv_ksize

    def unfolding(self, x: Tensor) -> Tuple[Tensor, Dict]:
        patch_w, patch_h = self.patch_w, self.patch_h
        patch_area = patch_w * patch_h
        batch_size, in_channels, orig_h, orig_w = x.shape

        new_h = int(math.ceil(orig_h / self.patch_h) * self.patch_h)
        new_w = int(math.ceil(orig_w / self.patch_w) * self.patch_w)

        interpolate = False
        if new_w != orig_w or new_h != orig_h:
            # Note: Padding can be done, but then it needs to be handled in attention function.
            x = ops.interpolate(x, size=(new_h, new_w), coordinate_transformation_mode="align_corners", mode="bilinear")
            interpolate = True

        # number of patches along width and height
        num_patch_w = new_w // patch_w  # n_w
        num_patch_h = new_h // patch_h  # n_h
        num_patches = num_patch_h * num_patch_w  # N
        
        # [B, C, H, W] -> [B * C * n_h, p_h, n_w, p_w]
        x = ops.reshape(x, (batch_size * in_channels * num_patch_h, patch_h, num_patch_w, patch_w))
        # [B * C * n_h, p_h, n_w, p_w] -> [B * C * n_h, n_w, p_h, p_w]
        x = ops.transpose(x, (0, 2, 1, 3))
        # [B * C * n_h, n_w, p_h, p_w] -> [B, C, N, P] where P = p_h * p_w and N = n_h * n_w
        x = ops.reshape(x, (batch_size, in_channels, num_patches, patch_area))
        # [B, C, N, P] -> [B, P, N, C]
        x = ops.transpose(x, (0, 3, 2, 1))
        # [B, P, N, C] -> [BP, N, C]
        x = ops.reshape(x, (batch_size * patch_area, num_patches, -1))

        info_dict = {
            "orig_size": (orig_h, orig_w),
            "batch_size": batch_size,
            "interpolate": interpolate,
            "total_patches": num_patches,
            "num_patches_w": num_patch_w,
            "num_patches_h": num_patch_h,
        }

        return x, info_dict

    def folding(self, x: Tensor, info_dict: Dict) -> Tensor:
        n_dim = ops.rank(x)
        assert n_dim == 3, "Tensor should be of shape BPxNxC. Got: {}".format(
            x.shape
        )
        # [BP, N, C] --> [B, P, N, C]
        x = x.view(
            info_dict["batch_size"], self.patch_area, info_dict["total_patches"], -1
        )

        batch_size, pixels, num_patches, channels = x.shape
        num_patch_h = info_dict["num_patches_h"]
        num_patch_w = info_dict["num_patches_w"]

        # [B, P, N, C] -> [B, C, N, P]
        x = ops.transpose(x, (0, 3, 2, 1))
        # [B, C, N, P] -> [B*C*n_h, n_w, p_h, p_w]
        x = ops.reshape(x, (batch_size * channels * num_patch_h, num_patch_w, self.patch_h, self.patch_w))
        # [B*C*n_h, n_w, p_h, p_w] -> [B*C*n_h, p_h, n_w, p_w]
        x = ops.transpose(x, (0, 2, 1, 3))
        # [B*C*n_h, p_h, n_w, p_w] -> [B, C, H, W]
        x = ops.reshape(x, (batch_size, channels, num_patch_h * self.patch_h, num_patch_w * self.patch_w))
        if info_dict["interpolate"]:
            x = ops.interpolate(
                x,
                size=info_dict["orig_size"],
                coordinate_transformation_mode="align_corners",
                mode="bilinear",
            )
        return x

    def construct(self, x: Tensor) -> Tensor:
        res = x
        fm = self.local_rep(x)
        # convert feature map to patches
        patches, info_dict = self.unfolding(fm) # 将(B, C, H, W)的输入展开成(B*P, N, C), 其中P = (h*w), h,w 是image patch 的高和宽，N = (H*W)/P是image patch的数量 
        # learn global representations
        for transformer_layer in self.global_rep: # transformer 学习全局信息
            patches = transformer_layer(patches)
        # [B x Patch x Patches x C] -> [B x C x Patches x Patch]
        fm = self.folding(x=patches, info_dict=info_dict) # 将(B*P, N, C)的特征折叠成 (B, C, H, W)
        fm = self.conv_proj(fm)
        fm = self.fusion(ops.concat((res, fm), 1))
        return fm

## ViT 实战 <a class="anchor" id="second-sec"></a>

接下来，我们使用MindCV进行ViT图片分类的实战练习。实战练习的文档参考[这里](./ViT_image_classification_tutorial.ipynb)。

